In [2]:
import pandas as pd
import requests

geoapify_key = "3a97a8ac79c943f98f95319fc9d33fcb"

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv('2023040-citibike-tripdata.csv')

# Extract unique station coordinates
unique_stations = data[['start_lat', 'start_lng']].drop_duplicates()

# Initialize an empty list to store the zip codes
zip_codes = []

/var/folders/pf/44qwd9f54h7f1cqwhnnmjrjr0000gn/T/ipykernel_5058/914452680.py:7: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('2023040-citibike-tripdata.csv')


In [4]:
# Iterate over the unique stations and make API calls
for _, row in unique_stations.iterrows():
    lat, lng = row['start_lat'], row['start_lng']
    
    # Make an API request to Geoapify Geocoding API
    response = requests.get(f'https://api.geoapify.com/v1/geocode/reverse?lat={lat}&lon={lng}&apiKey={geoapify_key}')
    
    # Extract the zip code from the API response
    if response.status_code == 200:
        data = response.json()
        zip_code = data['features'][0]['properties']['postcode']
        zip_codes.append(zip_code)
    else:
        zip_codes.append(None)  # Handle the case when API request fails or no zip code is found

KeyError: 'postcode'

In [ ]:
# Add the zip codes to the DataFrame
unique_stations['zip_code'] = zip_codes

# Merge the zip code information back to the original data
data_with_zip = pd.merge(data, unique_stations, on=['latitude', 'longitude'], how='left')

# Save the modified DataFrame to a new CSV file
data_with_zip.to_csv('2023040-citibike-tripdata.csv', index=False)